In [15]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sachsene/carla-traffic-signs-images")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\julia\.cache\kagglehub\datasets\sachsene\carla-traffic-signs-images\versions\1


In [16]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Num GPUs Available: ", torch.cuda.device_count())

Num GPUs Available:  0


In [17]:
# Paths to the dataset
train_dataset_path = os.path.join("traffic_signs_images", "train")
test_dataset_path = os.path.join("traffic_signs_images", "test")

# Define the transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(32, scale=(0.9, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the training dataset with validation split
train_dataset = datasets.ImageFolder(train_dataset_path, transform=transform)
test_dataset = datasets.ImageFolder(test_dataset_path, transform=transform)


# Number and name of classes
num_classes = len(train_dataset.classes)
print(f'Number of classes: {num_classes}')
print('Classes:', train_dataset.classes)


# Split indices for training and validation (90% train, 10% validation)
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_indices, val_indices = torch.utils.data.random_split(train_dataset, [train_size, val_size])

# Create samplers for the training and validation datasets
train_sampler = SubsetRandomSampler(train_indices.indices)
val_sampler = SubsetRandomSampler(val_indices.indices)

# DataLoader for training, validation, and testing datasets
train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(train_dataset, batch_size=32, sampler=val_sampler)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Number of classes: 8
Classes: ['back', 'speed_30', 'speed_60', 'speed_90', 'speed_limit_30', 'speed_limit_40', 'speed_limit_60', 'stop']


In [18]:
# Define the CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)  # Adjusted for increased capacity
        self.fc2 = nn.Linear(128, 8)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Conv2D -> ReLU -> MaxPool
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)  # Flatten
        x = F.relu(self.fc1(x))  # Dense layer
        x = self.fc2(x)  # Output layer with softmax activation
        return F.log_softmax(x, dim=1)


In [19]:
# Instantiate model, loss function, and optimizer
model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Early stopping and model checkpoint functionality
best_val_acc = 0
patience = 50
epochs_no_improve = 0

# Training loop
num_epochs = 200

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = correct / total

    # Validation step
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = val_correct / val_total

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader):.4f}, '
          f'Train Acc: {train_acc:.4f}, Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {val_acc:.4f}')

    # Early stopping based on validation accuracy
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'BEST_Traffic_Sign_Model.pth')  # Save best model
    else:
        epochs_no_improve += 1

    if epochs_no_improve == patience:
        print(f'Early stopping after {epoch+1} epochs')
        break

Epoch 1/200, Train Loss: 1.2388, Train Acc: 0.5277, Val Loss: 0.7094, Val Acc: 0.6971
Epoch 2/200, Train Loss: 0.6488, Train Acc: 0.7177, Val Loss: 0.4877, Val Acc: 0.7886
Epoch 3/200, Train Loss: 0.5195, Train Acc: 0.7997, Val Loss: 0.3517, Val Acc: 0.9029
Epoch 4/200, Train Loss: 0.3011, Train Acc: 0.9034, Val Loss: 0.2390, Val Acc: 0.8914
Epoch 5/200, Train Loss: 0.2259, Train Acc: 0.9218, Val Loss: 0.1709, Val Acc: 0.9600
Epoch 6/200, Train Loss: 0.1777, Train Acc: 0.9421, Val Loss: 0.0918, Val Acc: 0.9771
Epoch 7/200, Train Loss: 0.0841, Train Acc: 0.9746, Val Loss: 0.0578, Val Acc: 0.9886
Epoch 8/200, Train Loss: 0.0709, Train Acc: 0.9803, Val Loss: 0.0621, Val Acc: 0.9771
Epoch 9/200, Train Loss: 0.0543, Train Acc: 0.9822, Val Loss: 0.0580, Val Acc: 0.9771
Epoch 10/200, Train Loss: 0.0708, Train Acc: 0.9797, Val Loss: 0.0794, Val Acc: 0.9714
Epoch 11/200, Train Loss: 0.0600, Train Acc: 0.9803, Val Loss: 0.0934, Val Acc: 0.9829
Epoch 12/200, Train Loss: 0.0408, Train Acc: 0.9892,

In [20]:
# Evaluate the model on the test set
model.load_state_dict(torch.load('BEST_Traffic_Sign_Model.pth'))  # Load best model
model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_acc = test_correct / test_total
print(f'Test accuracy: {test_acc:.4f}')

C:\Users\julia\AppData\Local\Temp\ipykernel_36304\3880124287.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('BEST_Traffic_Sign_Model.pt

Test accuracy: 0.9505


In [22]:
# Save the final model
torch.save(model.state_dict(), 'traffic_Sign_Model.pth')